In [ ]:
# Step 1: Setup and Imports
import requests
import pandas as pd
import openai
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve API keys from environment variables
VIRUSTOTAL_API_KEY = os.getenv("VIRUSTOTAL_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# Set the OpenAI API configuration
openai.api_key = OPENAI_API_KEY

# Step 2: Fetching Data from VirusTotal
def fetch_malware_data(api_key, limit=1000):
    url = 'https://www.virustotal.com/api/v3/files'
    headers = {'x-apikey': api_key}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()['data']
    else:
        raise Exception("Failed to fetch data: " + response.text)

malware_samples = fetch_malware_data(VIRUSTOTAL_API_KEY)



In [ ]:
# Step 3: Data Preprocessing
# Extract malware names and their descriptions
data = []
for sample in malware_samples:
    name = sample['attributes']['meaningful_name']
    malware_type = sample['attributes']['type_description']
    data.append({'name': name, 'type': malware_type})

df = pd.DataFrame(data)

In [ ]:
# Step 4: Training the Model with OpenAI
# Define the prompt template
def create_prompt(name):
    return f"Given the file name '{name}', classify its malware type."

# Create training data
training_data = [create_prompt(row['name']) + f" The type is: {row['type']}" for index, row in df.iterrows()]

# Train the model
train_response = openai.FineTune.create(
    training_file=training_data,
    model="gpt-3.5-turbo",
    n_epochs=5
)

In [ ]:
# Step 5: Testing the Model
# Test with a sample file name
test_name = "example_virus.exe"
prompt = create_prompt(test_name)
response = openai.Completion.create(
    model="gpt-3.5-turbo",
    prompt=prompt,
    max_tokens=50
)

print("Predicted malware type:", response['choices'][0]['text'].strip())

